In [1]:
# from pymatgen.core import Structure
# from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
# from workflow_utils import *
from distortion_analysis import *
# import os
# import pandas as pd
import glob

## Extract modes info (irreps, symmetry)

In [2]:
subdir = 'data/phon'
save_loc = f'{subdir}/irreps_std.json'

### Check if we already parsed
if os.path.exists(save_loc):
    df = pd.read_json(save_loc)
### Parse and save (bc it's kinda slow)
else:
    irreps_file = 'irreps_std.yaml'
    df = parse_irreps(subdir, irreps_file, dim_str='2 2 2', kpt='0 0 0')
    df.to_json(save_loc)
df

,band_indices,frequency,characters,point_group,is_polar,potential_fe,multiplicity
0,[1],-3.541120,"[[1, 0.0], [1, 1.8]]",6mm,True,True,1
1,[2],-1.253762,"[[1, 0.0], [1, 181.8]]",-3m,False,False,1
2,[3],0.091574,"[[1, 0.0], [1, 1.8]]",m,True,False,1
3,[4],0.099102,"[[1, 0.0], [1, 181.8]]",m,True,False,1
4,[5],0.239329,"[[1, 0.0], [1, 1.8]]",6mm,True,False,1
5,[6],0.319599,"[[1, 0.0], [1, 1.8]]",2/m,False,False,1
6,[7],0.391769,"[[1, 0.0], [1, 181.8]]",2/m,False,False,1
7,[8],2.674690,"[[1, 0.0], [1, 181.8]]",-3m,False,False,1
8,"[9, 10]",4.309331,"[[2, 0.0], [0, 178.2]]",2,False,False,2
9,[11],4.825129,"[[1, 0.0], [1, 181.8]]",m,True,False,1


## Generate modulated structures

In [3]:
os.getcwd()

'/home/kdmiller/Projects/distortion_analysis'

In [4]:
subdir = 'data/phon'
irreps_file = 'irreps_std.yaml'

### Define the dimensions of the modulation
### NB: The gen_mod_strucs() function converts to primitive so these dimensions can be larger than needed but they could be, for example 1,1,1 if you're only modulating G-pt modes
moddims = [2,2,2]

### Detail the kpoints, indices, and displacements of modes you want to modulate 
label_ls = [['G'],['G'],['X'],['Z'],['Z'],['G','G'],['Z','Z']]
indices_ls = [[1],[2],[1],[1],[2],[1,2],[1,2]]
disps_ls = [[1 for label in labels] for labels in label_ls]

### Syntactic sugar for defining the kpoints/labels for this structure
kpts_dict = {'G':'0 0 0', 'X':'0.5 0 0', 'Z':'0.5 0.5 -0.5'}
kpts_ls = [[kpts_dict[label] for label in labels] for labels in label_ls]

### Detail the locations of your phonon calculation directories
phon_dirs = glob.glob('data/phon')
print(phon_dirs)
phon_dirs = [os.path.abspath(p) for p in phon_dirs]

### For each phonon directory, generate the modulated structures
for phon_dir in phon_dirs:
    print(phon_dir)
    ### Modulate structures for each desired mode
    for i in range(len(label_ls)):
        struc, path = gen_mod_struc(phon_dir, label_ls[i], kpts_ls[i], indices_ls[i], disps_ls[i], moddims)
        print(path)
        print(struc)
        os.makedirs(f'{path}/processed_strucs', exist_ok=True)
        struc.to(f'{path}/processed_strucs/MPOSCAR.vasp', 'poscar')

['data/phon']
/home/kdmiller/Projects/distortion_analysis/data/phon
        _
  _ __ | |__   ___  _ __   ___   _ __  _   _
 | '_ \| '_ \ / _ \| '_ \ / _ \ | '_ \| | | |
 | |_) | | | | (_) | | | | (_) || |_) | |_| |
 | .__/|_| |_|\___/|_| |_|\___(_) .__/ \__, |
 |_|                            |_|    |___/
                                      2.19.1

Python version 3.10.10
Spglib version 2.0.2

"mod.conf" was read as phonopy configuration file.
Crystal structure was read from "phonopy_disp.yaml".
Unit of length: angstrom
Modulation mode
Settings:
  Force constants symmetrization: on
  Supercell: [2 2 2]
Spacegroup: P6_3/mmc (194)
Use -v option to watch primitive cell, unit cell, and supercell structures.

Force sets were not found in "phonopy_disp.yaml".
Forces and displacements were read from "FORCE_SETS".
Computing force constants...
Max drift of force constants: -0.319412 (xx) -0.000004 (xx)
Max drift after symmetrization by translation: -0.000000 (xx) -0.000000 (xx)

Modulated struc

## Generate interpolations between distorted structures


In [5]:
### Import and sort structures

centro = Structure.from_file('data/struc_centro.vasp')
centro = sort_sites(centro)
sga = SpacegroupAnalyzer(centro)
print(sga.get_space_group_symbol())
print(centro)

p = Structure.from_file('data/struc_plusP.vasp')
p = sort_sites(p)
sga = SpacegroupAnalyzer(p)
print(sga.get_space_group_symbol())
print(p)

R-3c
Full Formula (Ti2 Mn2 O6)
Reduced Formula: TiMnO3
abc   :   5.546368   5.546368   5.546368
angles:  55.670631  55.670631  55.670631
pbc   :       True       True       True
Sites (10)
  #  SP          a        b        c
---  ----  -------  -------  -------
  0  Ti    0.02202  0.02202  0.02202
  1  Ti    0.52202  0.52202  0.52202
  2  Mn    0.27202  0.27202  0.27202
  3  Mn    0.77202  0.77202  0.77202
  4  O     0.77202  0.39678  0.14726
  5  O     0.64726  0.89678  0.27202
  6  O     0.14726  0.77202  0.39678
  7  O     0.89678  0.27202  0.64726
  8  O     0.39678  0.14726  0.77202
  9  O     0.27202  0.64726  0.89678
R3c
Full Formula (Ti2 Mn2 O6)
Reduced Formula: TiMnO3
abc   :   5.546368   5.546368   5.546368
angles:  55.670631  55.670631  55.670631
pbc   :       True       True       True
Sites (10)
  #  SP          a        b        c
---  ----  -------  -------  -------
  0  Ti    0.00039  0.00039  0.00039
  1  Ti    0.50039  0.50039  0.50039
  2  Mn    0.21719  0.21719  0.

In [6]:
### Generate 10 distorted structures between the centro and plusP structures and some structures beyond it to create a nice looking Landau plot
end_amplitude = 1.3
increment = 0.1
strucs = centro.interpolate(p, interpolate_lattices=True, 
                            nimages=int((end_amplitude+0.00001)//increment),
                            end_amplitude=end_amplitude)
### Examine our results
for i in [0,5,10,13]:
    print(f'\nAmplitude = {i*increment}')
    print(strucs[i].frac_coords-centro.frac_coords)


Amplitude = 0.0
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

Amplitude = 0.5
[[-0.010815   -0.010815   -0.010815  ]
 [-0.01081499 -0.01081499 -0.01081499]
 [-0.02741499 -0.02741499 -0.02741499]
 [-0.02741498 -0.02741498 -0.02741498]
 [ 0.00474    -0.00237    -0.00237   ]
 [-0.00237    -0.00237     0.00474   ]
 [-0.00237     0.00474    -0.00237   ]
 [-0.00237     0.00474    -0.00237   ]
 [-0.00237    -0.00237     0.00474   ]
 [ 0.00474    -0.00237    -0.00237   ]]

Amplitude = 1.0
[[-0.02163    -0.02163    -0.02163   ]
 [-0.02162999 -0.02162999 -0.02162999]
 [-0.05482998 -0.05482998 -0.05482998]
 [-0.05482995 -0.05482995 -0.05482995]
 [ 0.00948    -0.00474    -0.00474   ]
 [-0.00474    -0.00474     0.00948   ]
 [-0.00474     0.00948    -0.00474   ]
 [-0.00474     0.00948    -0.00474   ]
 [-0.00474    -0.00474     0.00948   ]
 [ 0.00948    -0.00474    -0.00474   ]]

Amplitude = 1.3
[[-0.028119   -0.028119   -0.

## Generate a -P-distorted structure from a centrosymmetric and +P-distorted structure

In [7]:
### Import and sort structures

centro = Structure.from_file('data/struc_centro.vasp')
centro = sort_sites(centro)
print(centro)

p = Structure.from_file('data/struc_plusP.vasp')
p = sort_sites(p)
print(p)

Full Formula (Ti2 Mn2 O6)
Reduced Formula: TiMnO3
abc   :   5.546368   5.546368   5.546368
angles:  55.670631  55.670631  55.670631
pbc   :       True       True       True
Sites (10)
  #  SP          a        b        c
---  ----  -------  -------  -------
  0  Ti    0.02202  0.02202  0.02202
  1  Ti    0.52202  0.52202  0.52202
  2  Mn    0.27202  0.27202  0.27202
  3  Mn    0.77202  0.77202  0.77202
  4  O     0.77202  0.39678  0.14726
  5  O     0.64726  0.89678  0.27202
  6  O     0.14726  0.77202  0.39678
  7  O     0.89678  0.27202  0.64726
  8  O     0.39678  0.14726  0.77202
  9  O     0.27202  0.64726  0.89678
Full Formula (Ti2 Mn2 O6)
Reduced Formula: TiMnO3
abc   :   5.546368   5.546368   5.546368
angles:  55.670631  55.670631  55.670631
pbc   :       True       True       True
Sites (10)
  #  SP          a        b        c
---  ----  -------  -------  -------
  0  Ti    0.00039  0.00039  0.00039
  1  Ti    0.50039  0.50039  0.50039
  2  Mn    0.21719  0.21719  0.21719
  3

In [8]:
### Examine difference between structures to check for errors
print('+P distortions')
print(centro.frac_coords - p.frac_coords)

+P distortions
[[ 0.02163     0.02163     0.02163   ]
 [ 0.02162999  0.02162999  0.02162999]
 [ 0.05482998  0.05482998  0.05482998]
 [ 0.05482995  0.05482995  0.05482995]
 [-0.00948     0.00474     0.00474   ]
 [ 0.00474     0.00474    -0.00948   ]
 [ 0.00474    -0.00948     0.00474   ]
 [ 0.00474    -0.00948     0.00474   ]
 [ 0.00474     0.00474    -0.00948   ]
 [-0.00948     0.00474     0.00474   ]]


In [9]:
### Generate the mirror image, check that differences are opposite of plusP
_, minusP = centro.interpolate(p, end_amplitude=-1, nimages=1)
print('-P distortions')
print(centro.frac_coords - minusP.frac_coords)
print('\n-P + +P = 0')
print((centro.frac_coords - minusP.frac_coords) + (centro.frac_coords - p.frac_coords))
### Save the -P structure
minusP.to_file(filename='data/struc_minusP.vasp', fmt='poscar')

-P distortions
[[-0.02163    -0.02163    -0.02163   ]
 [-0.02162999 -0.02162999 -0.02162999]
 [-0.05482998 -0.05482998 -0.05482998]
 [-0.05482995 -0.05482995 -0.05482995]
 [ 0.00948    -0.00474    -0.00474   ]
 [-0.00474    -0.00474     0.00948   ]
 [-0.00474     0.00948    -0.00474   ]
 [-0.00474     0.00948    -0.00474   ]
 [-0.00474    -0.00474     0.00948   ]
 [ 0.00948    -0.00474    -0.00474   ]]

-P + +P = 0
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


'Ti2 Mn2 O6\n1.0\n   5.5463676453000001    0.0000000000000000    0.0000000000000000\n   3.1278707900999998    4.5802421745000004    0.0000000000000000\n   3.1278707900999998    1.6516038631000001    4.2720982029999996\nTi Mn O\n2 2 6\ndirect\n   0.0436499980000000    0.0436499980000000    0.0436499980000000 Ti\n   0.5436499710000000    0.5436499710000000    0.5436499710000000 Ti\n   0.3268499969999999    0.3268499969999999    0.3268499969999999 Mn\n   0.8268499370000000    0.8268499370000000    0.8268499370000000 Mn\n   0.7625399819999999    0.4015200140000000    0.1520000100000000 O\n   0.6520000100000000    0.9015200139999999    0.2625400119999999 O\n   0.1520000100000000    0.7625399819999999    0.4015200140000000 O\n   0.9015200139999999    0.2625400119999999    0.6520000100000000 O\n   0.4015200140000000    0.1520000100000000    0.7625399819999999 O\n   0.2625400119999999    0.6520000100000000    0.9015200139999999 O\n'